# Imports and Setup

In [1]:
!pip uninstall -y flwr ray
!pip install -U "flwr[simulation]" ray
!pip install pyannote.metrics
!pip install pyannote.core
!pip install speechbrain
!pip install datasets

Found existing installation: flwr 1.15.2
Uninstalling flwr-1.15.2:
  Successfully uninstalled flwr-1.15.2
Found existing installation: ray 2.31.0
Uninstalling ray-2.31.0:
  Successfully uninstalled ray-2.31.0
  Using cached ray-2.42.1-cp311-cp311-manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached flwr-1.15.2-py3-none-any.whl.metadata (15 kB)
  Using cached ray-2.31.0-cp311-cp311-manylinux2014_x86_64.whl.metadata (13 kB)
Using cached ray-2.31.0-cp311-cp311-manylinux2014_x86_64.whl (66.7 MB)
Using cached flwr-1.15.2-py3-none-any.whl (531 kB)


In [2]:
import os
import pickle
from collections import OrderedDict
from typing import List, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import librosa
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, Dataset, random_split
import flwr as fl
from flwr.client import NumPyClient
from flwr.common import Context
from pyannote.core import Segment, Annotation
from pyannote.metrics.diarization import DiarizationErrorRate
from speechbrain.pretrained import EncoderClassifier

torch.cuda.empty_cache()

# **Set device**
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {DEVICE}")

/usr/local/lib/python3.11/dist-packages/pyannote/core/notebook.py:134: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cm = get_cmap("Set1")


Training on cpu


<ipython-input-2-9f74519d6701>:21: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier


# Load and Process Data

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read)

In [4]:
def power_set_encoding(labels):
    """Encodes speaker pairs into a fixed label mapping."""
    if not labels or not isinstance(labels, list):
        print(f"❌ Invalid labels: {labels}")  # Debug
        return 0  # If no one is speaking, return 0

    # ✅ Standardize sorting to avoid different orders
    labels = sorted(labels)

    # ✅ Define the mapping explicitly
    encoding_map = {
        tuple(): 0,     # No one speaking
        ("A", "A"): 1,  # Speaker A alone
        ("B", "B"): 2,  # Speaker B alone
        ("A", "B"): 3   # Both speakers
    }

    # 🔹 Convert list to tuple for lookup
    encoded_label = encoding_map.get(tuple(labels), -1)

    # ✅ If label is not found, print error and return fallback
    if encoded_label == -1:
        print(f"❌ Unexpected speaker combination: {labels}, setting to 0")
        encoded_label = 0  # Default to silence if unexpected case appears

    return encoded_label

In [5]:
# Load preprocessed dataset
with open("preprocessed_callhome_data.pkl", "rb") as f:
    preprocessed_data = pickle.load(f)
extracted_segments = preprocessed_data.get("extracted_segments", [])
print(f"Total extracted segments: {len(extracted_segments)}")

Total extracted segments: 13358


In [6]:
# Collect unique speaker labels
unique_speaker_labels = set()

for seg in extracted_segments:
    unique_speaker_labels.update(seg["speakers"])

print(f"🔍 Unique speaker labels in dataset: {unique_speaker_labels}")

🔍 Unique speaker labels in dataset: {'B2', 'A1', 'A', 'B', 'B1'}


In [7]:
for seg in extracted_segments:
    seg["speakers"] = ["A" if s == "A1" else "B" if s == "B1" or s == "B2" else s for s in seg["speakers"]]

In [8]:
# Collect unique speaker labels
unique_speaker_labels = set()

for seg in extracted_segments:
    unique_speaker_labels.update(seg["speakers"])

print(f"🔍 Unique speaker labels in dataset: {unique_speaker_labels}")

🔍 Unique speaker labels in dataset: {'A', 'B'}


In [9]:
# ✅ Debug: Check if all extracted segments have valid labels
for i, seg in enumerate(extracted_segments[:50]):  # First 10 samples
    label = power_set_encoding(seg["speakers"])
    print(f"🔍 Segment {i} Speakers: {seg['speakers']}, Encoded Label: {label}")

🔍 Segment 0 Speakers: ['A', 'B'], Encoded Label: 3
🔍 Segment 1 Speakers: ['B', 'A'], Encoded Label: 3
🔍 Segment 2 Speakers: ['A', 'B'], Encoded Label: 3
🔍 Segment 3 Speakers: ['A', 'B'], Encoded Label: 3
🔍 Segment 4 Speakers: ['B', 'A'], Encoded Label: 3
🔍 Segment 5 Speakers: ['A', 'B'], Encoded Label: 3
🔍 Segment 6 Speakers: ['B', 'A'], Encoded Label: 3
🔍 Segment 7 Speakers: ['B', 'A'], Encoded Label: 3
🔍 Segment 8 Speakers: ['B', 'A'], Encoded Label: 3
🔍 Segment 9 Speakers: ['A', 'B'], Encoded Label: 3
🔍 Segment 10 Speakers: ['A', 'B'], Encoded Label: 3
🔍 Segment 11 Speakers: ['A', 'B'], Encoded Label: 3
🔍 Segment 12 Speakers: ['B', 'A'], Encoded Label: 3
🔍 Segment 13 Speakers: ['B', 'A'], Encoded Label: 3
🔍 Segment 14 Speakers: ['A', 'B'], Encoded Label: 3
🔍 Segment 15 Speakers: ['B', 'A'], Encoded Label: 3
🔍 Segment 16 Speakers: ['B', 'A'], Encoded Label: 3
🔍 Segment 17 Speakers: ['A', 'B'], Encoded Label: 3
🔍 Segment 18 Speakers: ['B', 'A'], Encoded Label: 3
🔍 Segment 19 Speakers:

In [10]:
from collections import Counter

all_speaker_labels = [tuple(sorted(seg["speakers"])) for seg in extracted_segments]
speaker_count = Counter(all_speaker_labels)

print("\n🔍 Unique speaker combinations with their frequencies:")
for speakers, count in speaker_count.items():
    print(f"{speakers}: {count} occurrences")


🔍 Unique speaker combinations with their frequencies:
('A', 'B'): 13279 occurrences
('B', 'B'): 43 occurrences
('A', 'A'): 36 occurrences


In [11]:
print("\n🔍 How often do 'A1' and 'B1' appear separately or together?")
for speakers, count in speaker_count.items():
    if "A" in speakers or "B" in speakers:
        print(f"{speakers}: {count} occurrences")


🔍 How often do 'A1' and 'B1' appear separately or together?
('A', 'B'): 13279 occurrences
('B', 'B'): 43 occurrences
('A', 'A'): 36 occurrences


In [12]:
# Load dataset for retrieving full audio samples
from datasets import load_dataset
dataset = load_dataset("talkbank/callhome", "eng", split="data")
print(f"Original dataset contains {len(dataset)} samples.")

# **Feature Extraction Function**
def extract_log_mel_spectrogram(segment):
    """Extracts Log-Mel Spectrogram features for input to SEND model."""
    source_idx = segment.get("source")
    if source_idx is None or source_idx < 0 or source_idx >= len(dataset):
        raise ValueError("Invalid source index in segment.")

    sample = dataset[source_idx]
    audio_info = sample["audio"]
    audio_array = audio_info["array"]
    sr = audio_info["sampling_rate"]

    start_time = segment["timestamp_start"]
    end_time = segment["timestamp_end"]
    start_sample = int(start_time * sr)
    end_sample = int(end_time * sr)
    snippet = audio_array[start_sample:end_sample]

    # Pad if too short
    min_length = sr // 10  # 100ms minimum
    if len(snippet) < min_length:
        snippet = np.pad(snippet, (0, min_length - len(snippet)), mode='constant')

    # Compute Log-Mel Spectrogram
    mel_spec = librosa.feature.melspectrogram(y=snippet, sr=sr, n_mels=80)
    log_mel = librosa.power_to_db(mel_spec)
    log_mel = torch.tensor(log_mel.T, dtype=torch.float32)  # Shape: [time, features]

    return log_mel

# **Prepare dataset for training**
segment_features = []
segment_labels = []
for seg in extracted_segments:
    try:
        feat = extract_log_mel_spectrogram(seg)
        label = power_set_encoding(seg["speakers"])
        segment_features.append(feat)
        segment_labels.append(label)
    except Exception as e:
        print(f"Skipping segment due to error: {e}")

print(f"Total segments with features: {len(segment_features)}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Original dataset contains 140 samples.


/usr/local/lib/python3.11/dist-packages/librosa/core/intervals.py:15: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with resources.path("librosa.core", "intervals.msgpack") as imsgpack:
/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1600
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1920
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1921
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1761
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/librosa/core/spe

Total segments with features: 13358


# Define Dataset and DataLoader

In [13]:
# **Dataset Class**
class OverlappingSpeechDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if idx >= len(self.features) or self.features[idx] is None or self.labels[idx] is None:
            print(f"⚠️ Warning: Invalid sample at index {idx}. Feature or Label is None.")
            return None  # Prevent errors

        feature = self.features[idx]
        label = self.labels[idx]

        print(f"✅ Sample {idx}: Feature Shape: {feature.shape}, Label: {label}")

        return feature, torch.tensor(label, dtype=torch.long)

from torch.nn.utils.rnn import pad_sequence

def collate_fn_pad(batch):
    """Pads variable-length sequences to the longest in the batch."""
    batch = [b for b in batch if b is not None]  # Remove None samples

    if len(batch) == 0:
        return None, None  # Return empty batch if all are invalid

    features, labels = zip(*batch)

    # Pad sequences to the maximum length in this batch
    features_padded = pad_sequence(features, batch_first=True, padding_value=0)
    labels = torch.tensor(labels, dtype=torch.long)

    return features_padded, labels

# **Filter out None values before creating dataset**
filtered_features = [feat for feat, lbl in zip(segment_features, segment_labels) if feat is not None and lbl is not None]
filtered_labels = [lbl for feat, lbl in zip(segment_features, segment_labels) if feat is not None and lbl is not None]

# ✅ Debug: Check if filtering removed everything
print(f"✅ Filtered dataset size: {len(filtered_features)} samples")
print(f"✅ Filtered labels size: {len(filtered_labels)} labels")

if len(filtered_features) == 0:
    raise ValueError("🚨 No valid samples left after filtering! Check how power_set_encoding() assigns labels.")

# ✅ Print first few valid samples to verify
for i in range(min(5, len(filtered_features))):  # Print only available samples
    print(f"🔹 Sample {i}: Feature Shape: {filtered_features[i].shape}, Label: {filtered_labels[i]}")

# **Split dataset for Federated Learning**
train_val_size = int(0.9 * len(filtered_features))
final_test_size = len(filtered_features) - train_val_size

train_val_dataset, final_test_dataset = random_split(filtered_features, [train_val_size, final_test_size])
train_labels, test_labels = filtered_labels[:train_val_size], filtered_labels[train_val_size:]

# ✅ Debug: Ensure train/test splits are valid
print(f"✅ Train size: {len(train_val_dataset)}, Test size: {len(final_test_dataset)}")

batch_size = 16

train_loader = DataLoader(
    OverlappingSpeechDataset(train_val_dataset, train_labels),
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn_pad
)

test_loader = DataLoader(
    OverlappingSpeechDataset(final_test_dataset, test_labels),
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn_pad
)

✅ Filtered dataset size: 13358 samples
✅ Filtered labels size: 13358 labels
🔹 Sample 0: Feature Shape: torch.Size([11, 80]), Label: 3
🔹 Sample 1: Feature Shape: torch.Size([4, 80]), Label: 3
🔹 Sample 2: Feature Shape: torch.Size([12, 80]), Label: 3
🔹 Sample 3: Feature Shape: torch.Size([42, 80]), Label: 3
🔹 Sample 4: Feature Shape: torch.Size([8, 80]), Label: 3
✅ Train size: 12022, Test size: 1336


# Define SEND Model

In [14]:
num_speakers=2

# Federated Learning Client

In [15]:
print(f"🔍 Unique Encoded Labels: {set(segment_labels)}")

🔍 Unique Encoded Labels: {1, 2, 3}


In [16]:
import logging
import time
import flwr as fl

import importlib
import ray
importlib.reload(ray)

from speechbrain.pretrained import EncoderClassifier

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Define `speaker_encoder` correctly
speaker_encoder = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    savedir="pretrained_models/spkrec-ecapa",
    run_opts={"device": DEVICE}
).to(DEVICE)

# **SEND Model**
class SpeechEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)

    def forward(self, x):
        x, _ = self.lstm(x)
        return x

class SEND(nn.Module):
    def __init__(self, input_dim, speaker_dim, hidden_dim, num_layers, num_speakers):
        super().__init__()
        self.speech_encoder = SpeechEncoder(input_dim, hidden_dim, num_layers)
        self.speaker_encoder = speaker_encoder.encode_batch

        self.feature_projection = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.context_independent = nn.Linear(hidden_dim * 2, num_speakers)
        self.context_dependent = nn.TransformerEncoder(nn.TransformerEncoderLayer(hidden_dim * 2, 4), num_layers)
        self.match_projection = nn.Linear(hidden_dim * 2, num_speakers)
        self.post_net = nn.Linear(hidden_dim * 2, 2 ** num_speakers)  # Ensure input matches hidden_dim * 2

    def forward(self, speech):
        # Ensure correct shape
        if speech.dim() == 4:
            speech = speech.squeeze(1)

        speech_features = self.speech_encoder(speech)
        # print(f"🔹 Speech features shape after LSTM: {speech_features.shape}")  # [16, 29, 512]

        speech_features = self.feature_projection(speech_features)
        # print(f"🔹 Projected speech features shape: {speech_features.shape}")  # [16, 29, 512]

        # **Apply mean pooling across time dimension (sequence length 29)**
        pooled_features = torch.mean(speech_features, dim=1)  # [16, 512]
        # print(f"🔹 Pooled features shape: {pooled_features.shape}")  # [16, 512]

        # Ensure output matches batch size
        output = self.post_net(pooled_features)  # [16, num_classes]
        # print(f"🔹 Model output shape: {output.shape}")  # Should be [16, 2**num_speakers]

        return output

model = SEND(input_dim=80, speaker_dim=512, hidden_dim=256, num_layers=4, num_speakers=num_speakers).to(DEVICE)

class FLClient(NumPyClient):
    def __init__(self, model, train_loader, test_loader):
        os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # ✅ Force CPU globally
        self.device = torch.device("cpu")  # ✅ Force CPU for PyTorch
        logging.info(f"Using device: {self.device}")
        self.model = model.to(self.device)  # ✅ Move model to CPU
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.optimizer = optim.Adam(model.parameters(), lr=0.001)
        self.criterion = nn.CrossEntropyLoss()

    def get_parameters(self, config=None):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        state_dict = {k: torch.tensor(v).to(self.device) for k, v in zip(self.model.state_dict().keys(), parameters)}
        self.model.load_state_dict(state_dict)
        self.model.to(self.device)  # Ensure model is on the right device

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()

        logging.info(f"Training on {self.device}...")
        for epoch in range(1):
            for batch_idx, (features, labels) in enumerate(self.train_loader):
                features, labels = features.to(self.device), labels.to(self.device)  # ✅ Force CPU usage
                # print(f"Labels shape: {labels.shape}")  # Check shape
                # print(f"Sample labels: {labels[:5]}")  # Print first 5 labels
                self.optimizer.zero_grad()
                outputs = self.model(features)  # Should return (batch_size, num_classes)
                # print(f"Outputs shape: {outputs.shape}")  # Expected: [16, num_classes]
                # print(f"🔹 Labels shape before expansion: {labels.shape}")
                labels = labels.view(-1)  # Ensure correct shape
                # print(f"✅ Final Labels shape: {labels.shape}")  # Should be [16]

                loss = self.criterion(outputs, labels.long())  # Ensure it matches [16]
                loss.backward()
                self.optimizer.step()

        return self.get_parameters(), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()

        total_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_idx, (features, labels) in enumerate(self.test_loader):
                features, labels = features.to(self.device), labels.to(self.device)
                outputs = self.model(features)

                labels = labels.long()
                loss = self.criterion(outputs, labels)
                total_loss += loss.item()
                preds = torch.argmax(outputs, dim=-1)
                correct += (preds == labels).sum().item()
                total += labels.numel()

                # ✅ Add debug logging for evaluation
                logging.info(f"🔹 Evaluation Batch {batch_idx}")
                logging.info(f"🔹 Predictions: {preds[:5]}")
                logging.info(f"🔹 Ground Truth: {labels[:5]}")
                logging.info(f"🔹 Batch Loss: {loss.item()}")

        accuracy = correct / total if total > 0 else 0.0
        logging.info(f"✅ Evaluation Results - Loss: {total_loss / len(self.test_loader):.4f}, Accuracy: {accuracy:.2%}")
        return total_loss / len(self.test_loader), len(self.test_loader.dataset), {"accuracy": accuracy}

# ✅ Custom Strategy for Evaluation Debugging
class PrintEvaluateStrategy(fl.server.strategy.FedAvg):
    def aggregate_evaluate(self, rnd: int, results, failures):
        aggregated = super().aggregate_evaluate(rnd, results, failures)
        if aggregated is None:
            logging.info(f"[ROUND {rnd}] No evaluation results")
            return aggregated
        if isinstance(aggregated, tuple):
            if len(aggregated) == 3:
                loss, num_examples, metrics = aggregated
            elif len(aggregated) == 2:
                loss, num_examples = aggregated
                num_examples = num_examples if isinstance(num_examples, int) else "unknown"
                metrics = {}
            else:
                loss, num_examples, metrics = aggregated, "unknown", {}
        else:
            logging.info(f"[ROUND {rnd}] Aggregated evaluation: {aggregated}")
            return aggregated

        accuracy = metrics.get("accuracy", None)
        accuracy_str = f"{accuracy:.2%}" if accuracy is not None else "N/A"
        logging.info(f"[ROUND {rnd}] Evaluation results: Loss: {loss:.4f}, Accuracy: {accuracy_str} on {num_examples} examples")
        return aggregated

logging.basicConfig(level=logging.INFO)
strategy = PrintEvaluateStrategy(min_fit_clients=4, min_available_clients=4)

fl.simulation.start_simulation(
    client_fn=lambda ctx: FLClient(model, train_loader, test_loader),
    num_clients=4,
    config=fl.server.ServerConfig(num_rounds=2),
    strategy=fl.server.strategy.FedAvg(),
    ray_init_args={
        "num_cpus": 4,  # ✅ Only use CPU
        "include_dashboard": False,
        "ignore_reinit_error": True,
    },
    client_resources={"num_cpus": 1}  # ✅ No GPU needed
)

/usr/local/lib/python3.11/dist-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)
/usr/local/lib/python3.11/dist-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.seria

(ClientAppActor pid=63124) ✅ Sample 11492: Feature Shape: torch.Size([9, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 2257: Feature Shape: torch.Size([9, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 2415: Feature Shape: torch.Size([15, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 11027: Feature Shape: torch.Size([29, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 3328: Feature Shape: torch.Size([8, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 994: Feature Shape: torch.Size([18, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 6582: Feature Shape: torch.Size([9, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 7612: Feature Shape: torch.Size([46, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 11095: Feature Shape: torch.Size([23, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 11886: Feature Shape: torch.Size([23, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 7087: Feature Shape: torch.Size([4, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sampl

(ClientAppActor pid=63097) INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
(ClientAppActor pid=63097) INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
(ClientAppActor pid=63097) /usr/local/lib/python3.11/dist-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
(ClientAppActor pid=63097)   wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


(ClientAppActor pid=63124) ✅ Sample 8619: Feature Shape: torch.Size([12, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 5453: Feature Shape: torch.Size([4, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 11535: Feature Shape: torch.Size([5, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 358: Feature Shape: torch.Size([12, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 5627: Feature Shape: torch.Size([16, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 3156: Feature Shape: torch.Size([35, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 2020: Feature Shape: torch.Size([9, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 2880: Feature Shape: torch.Size([7, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 11456: Feature Shape: torch.Size([11, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 10145: Feature Shape: torch.Size([16, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sample 5766: Feature Shape: torch.Size([28, 80]), Label: 3
(ClientAppActor pid=63124) ✅ Sampl

(ClientAppActor pid=63043) INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
(ClientAppActor pid=63044) INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
(ClientAppActor pid=63043) INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): [] [repeated 2x across cluster]
(ClientAppActor pid=63044) /usr/local/lib/python3.11/dist-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
(ClientAppActor pid=63044)   wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)
(ClientAppActor pid=63043) /usr/local/lib/python3.11/dist-packages/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fw

...

(ClientAppActor pid=63043) ✅ Sample 1199: Feature Shape: torch.Size([36, 80]), Label: 3 [repeated 344x across cluster]


(ClientAppActor pid=63097) INFO:root:🔹 Evaluation Batch 74 [repeated 20x across cluster]
(ClientAppActor pid=63097) INFO:root:🔹 Predictions: tensor([3, 3, 3, 3, 3]) [repeated 20x across cluster]
(ClientAppActor pid=63097) INFO:root:🔹 Ground Truth: tensor([3, 3, 3, 3, 3]) [repeated 20x across cluster]
(ClientAppActor pid=63097) INFO:root:🔹 Batch Loss: 0.011568386107683182 [repeated 20x across cluster]


(ClientAppActor pid=63124) ✅ Sample 1231: Feature Shape: torch.Size([8, 80]), Label: 3 [repeated 240x across cluster]


(ClientAppActor pid=63043) INFO:root:✅ Evaluation Results - Loss: 0.0306, Accuracy: 99.63%
(ClientAppActor pid=63097) INFO:root:🔹 Evaluation Batch 79 [repeated 17x across cluster]
(ClientAppActor pid=63097) INFO:root:🔹 Predictions: tensor([3, 3, 3, 3, 3]) [repeated 17x across cluster]
(ClientAppActor pid=63097) INFO:root:🔹 Ground Truth: tensor([3, 3, 3, 3, 3]) [repeated 17x across cluster]
(ClientAppActor pid=63097) INFO:root:🔹 Batch Loss: 0.011604604311287403 [repeated 17x across cluster]
INFO :      aggregate_evaluate: received 4 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 2 round(s) in 5223.54s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.027965324088221506
INFO :      		round 2: 0.030552221428869025
INFO :      


History (loss, distributed):
	round 1: 0.027965324088221506
	round 2: 0.030552221428869025